In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from tqdm.auto import tqdm

In [2]:
from collections import Counter

In [3]:
transactions = pd.read_csv("../data/dunnhumby_The-Complete-Journey/dunnhumby - The Complete Journey CSV/transaction_data.csv")
products = pd.read_csv("../data/dunnhumby_The-Complete-Journey/dunnhumby - The Complete Journey CSV/product.csv")

In [4]:
#groups.drop_duplicates(subset=None)
merged = transactions.merge(products, how = 'inner', on = 'PRODUCT_ID')

In [5]:
merged["Name"] = merged["COMMODITY_DESC"] + " / " + merged["SUB_COMMODITY_DESC"]

In [6]:
essential_data = merged[["BASKET_ID", "Name"]]

In [ ]:
counts = Counter()
for _, products in tqdm(essential_data.groupby('BASKET_ID')["Name"]):
    products = np.unique(products)
    for i in range(0, len(products)):
        for j in range(0, i):
            a = products[i]
            b = products[j]
            if a < b:
                sa = a
                sb = b
            else:
                sa = b
                sb = a
            counts[a + ";" + b] += 1

In [ ]:
rows = []
for prods,count in tqdm(counts.items()):
    [a,b] = prods.split(";")
    rows.append((a,b,count))

In [23]:
counts_df = pd.DataFrame(rows, columns=["Product 1", "Product 2", "Count"])

In [24]:
counts_df = counts_df[counts_df["Count"] > 10]

In [25]:
counts_df.to_csv("named_counts_2.csv")

In [4]:
counts_df = pd.read_csv("../notebooks/named_counts_2.csv")

In [26]:
counts_df = counts_df.sort_values("Count", ascending=False)

## Introduction

We wanted to explore relations between different products - we decided to look for pairs of products that tend to be bought together frequently.

First we take a look at counts of transactions that both products appear in.

In [ ]:
counts_df.head(5)

In [ ]:
counts_df.iloc[4,0]

In [12]:
# the same table as above but formatted in a pretty way:

The 5 most often recurring pairs are:

<small>(We converted the original product names like "TROPICAL FRUIT / BANANAS" to simple, a bit less specific names for simplicity. We also show rounded counts in thousands of transactions.)</small>
    
| Product 1    | Product 2    | Co-occurence count |
|--------------|--------------|--------------------|
| Bananas       | Milk         |  ~16k             |
| Milk         | Bread      |  ~14k             |
| Some soft drink (can) | Milk         |  ~11k             |
| Shredded cheese  | Milk         |  ~10k             |
| Refrigeated juice       | Milk      |  ~15k             |


As we can see milk is present in almost every pair. This is likely to it being a frequently bought product overall.

In [35]:
popular = merged.groupby("Name").apply(len).sort_values(ascending=False).head(10)

In [ ]:
popular

Indeed, when we look at the top 5 most often bought products, we get almost the same results as for the top pairs:

| Product | Count |
|---------|-------|
| Milk    | ~66k   |
| Some soft drink (can) | ~39k |
| Other soft drink (2L bottle) | ~37k |
| Yogurt | ~37k |
| Bananas | ~30k |

Products that are more often bought overall are more likely to form the pairs of being bought together (because if both products have high probability of being bought in a transaction, the join probability is also higher than for products that are bough rarely overall).

To look further, we will look at relative counts - for each pair we will divide the co-occurrence count by the amount of transactions each of the products is in and take the bigger value. 

In [37]:
sales = merged.groupby(["Name"]).apply(lambda df: df["BASKET_ID"].nunique())

In [38]:
relativized = counts_df.merge(sales.rename("Sales 1"), left_on="Product 1", right_on="Name").merge(sales.rename("Sales 2"), left_on="Product 2", right_on="Name")

In [39]:
relativized["Relative Count 1"] = relativized["Count"] / relativized["Sales 1"]
relativized["Relative Count 2"] = relativized["Count"] / relativized["Sales 2"]

In [40]:
relativized["Relative Count"] = relativized[["Relative Count 1", "Relative Count 2"]].max(axis=1)

In [45]:
relativized_filt = relativized[relativized["Count"] > 100]

In [46]:
relativized_filt = relativized_filt.sort_values("Relative Count", ascending=False)

In [ ]:
relativized_filt.head(10)

## Relative count

Below we can see the top 5 pairs after relativizing the co-occurences. We have also removed pairs which were bought less than 100 times (because when one product was bought only once ever, it would result in 100% but that is not a useful conclusion but rather an outlier).

The interpretation of percentage is following: of all transactions where Primary product was bought, it shows the fraction of transactions where Secondary product was bought as well.

| Primary product | Secondary product | % of times secondary product was bought along the primary one | 
|-|-|-|
| Salad condiments (Salad bar) | Fresh fruit (Salad bar) | 91% |
| Fruit glazes | Strawberries | 73% |
| Instant breakfast | Milk | 73% | 
| Frozen pizza (some particular variant) | Frozen meat entree | 72% |
| Salad dressing mix | Sour cream | 71% |

For example, we can see that 91% of the times Salad condiments were bought at the salad bar, fresh fruit was also bought. This correlation seems quite likely as these are just 2 products from the salad bar, so they tend to be bought together quite often.

Then we can see that 73% of the time someone buys a fruit glaze, they also buy strawberries - of course they need something to put the glaze on, and strawberries look like a very common choice. Maybe it's a good idea for the retailers to do some promotions on buying these two together?

Then we can guess that the Instant breakfast is actually a kind of cereal package, as 73% of times it is bought it is bought with milk as well.

We can also see that people often buy sour cream along with the salad dressing. Also, many fans of frozen pizza also like an entree.

## Groups of products

In [ ]:
counts_df["Count"].plot.hist(bins=range(0, 2000, 50), log=True)

In [ ]:
(counts_df["Count"] > 1000).sum()

The next goal was to look at groups of products bigger than mere pairs.

As there are exponentially many subsets of all products we couldn't have counted them by brute-force, so we came with an approximation. We look at pairs of products that are bought together often (after looking at the distribution of co-occurence counts, we decided for a threshold of sharing at least 1000 transactions) and form a graph. Than we look for cliques in this graph - a clique is a set of products of which each pair was often bought together.

In [63]:
import networkx as nx

In [98]:
bigger_counts = counts_df[counts_df["Count"] > 3000]

In [99]:
graph = nx.from_pandas_edgelist(bigger_counts, 'Product 1', 'Product 2', edge_attr="Count", create_using= nx.Graph())

In [101]:
cliques = list(nx.clique.find_cliques(graph))

In [102]:
maxlen = max(map(len, cliques))

In [103]:
chosen_clique = None
chosen_w = None

In [ ]:
for clique in cliques:
    if len(clique) >= maxlen:
        weights = []
        for a in clique:
            for b in clique:
                if a != b:
                    weights.append(graph.get_edge_data(a,b)["Count"])
        print("Each pair has been bought together " + str(min(weights)) + " times:")
        if chosen_clique is None or min(weights) > chosen_w:
            chosen_w = min(weights)
            chosen_clique = clique

In [ ]:
chosen_clique

In [ ]:
def contains_clique(df):
    names = list(df["Name"])
    for p in chosen_clique:
        if p not in names:
            return False
    return True

In [ ]:
tqdm.pandas()

In [ ]:
containing = merged.groupby("BASKET_ID").progress_apply(contains_clique)

In [ ]:
containing.sum()

We have found that each pair of the following products has been bought together at least 3460 times:
 - Milk
 - Bananas
 - White bread
 - Shredded cheese
 - Potato chips
 - A soft drink in a can

There have been indeed 161 transactions that contained all of these items at once.

In [148]:
bigger_counts = relativized_filt[relativized_filt["Relative Count"] > 0.25]

In [149]:
graph = nx.from_pandas_edgelist(bigger_counts, 'Product 1', 'Product 2', edge_attr="Relative Count", create_using= nx.Graph())

In [ ]:
print(nx.info(graph))

In [151]:
cliques = list(nx.clique.find_cliques(graph))

In [152]:
maxlen = max(map(len, cliques))

In [153]:
chosen_clique = None
chosen_w = None

In [ ]:
for clique in cliques:
    if len(clique) >= maxlen:
        weights = []
        for a in clique:
            for b in clique:
                if a != b:
                    weights.append(graph.get_edge_data(a,b)["Relative Count"])
        print("Each pair has been bought together " + str(min(weights)) + " times:")
        print(clique)
        if chosen_clique is None or min(weights) > chosen_w:
            chosen_w = min(weights)
            chosen_clique = clique

In [ ]:
chosen_clique

In [ ]:
containing = merged.groupby("BASKET_ID").progress_apply(contains_clique)

In [ ]:
containing.sum()

As we can see Milk and Bananas in the list, we see that these products were just bought very often so it's also more likely for them to be bought together.

Because of that we decided to also look at the relativized percentages. We looked at cliques in a similar graph but based on pairs where the relative percentage was at least 25%.

In [155]:
for clique in cliques:
    if len(clique) >= maxlen: # select the first one
        chosen_clique = clique
        break

In [ ]:
chosen_clique

In [ ]:
containing = merged.groupby("BASKET_ID").progress_apply(contains_clique)

In [ ]:
containing.sum()

We have found out there were 32 transactions where all of:
- Milk
- Bananas
- Corn 
- Green beans
- Green peas
- Carrots
have been bought together. Looks like Milk and Bananas showed there mostly due to their very high popularity, but the rest look like common ingredients for a homemade salad.

Moreover, we found 47 transactions where all of:
- Milk
- Tortilla / nacho chips
- Shredded cheese
- Mexican salsa
- Mexican beans (refried)
- Mexican seasoning mix
have been bought. 

Of course 32 or 47 is very little transactions, but these are containing the full set of all these ingredients. The amount of transactions each subset will be significantly bigger.